My goal is to add the bytes-per-bit loss eval.

### Cache mapping from token id to number of bytes

I can see from the code I'll now need this but I skipped it when training the tokenizer. For now I'll do it right here in this notebook. Later I should move the token training and this into a `my_tok_train.py` script.

In [1]:
import sys
sys.path.append('../my_nanochat')
import os
import torch
from my_nanochat.my_tokenizer import get_tokenizer, get_token_bytes
from my_nanochat.my_common import get_base_dir

In [2]:
tokenizer = get_tokenizer()

In [3]:
tokenizer.get_special_tokens()

{'<bos>'}

^ This is the only special token for now because I didn't add the other ones yet. He adds many others but they only are needed later for finetuning.

Copying from the bottom of [tok_train.py](https://github.com/karpathy/nanochat/blob/master/scripts/tok_train.py)

In [4]:
vocab_size = tokenizer.get_vocab_size()
special_set = set(tokenizer.get_special_tokens())
token_strings = [tokenizer.decode([token_id]) for token_id in range(vocab_size)]
token_bytes = []
for token_id in range(vocab_size):
    token_str = token_strings[token_id]
    if token_str in special_set:
        token_bytes.append(0)
    else:
        id_bytes = len(token_str.encode("utf-8"))
        token_bytes.append(id_bytes)
token_bytes = torch.tensor(token_bytes, dtype=torch.int32, device="cpu")
token_bytes_path = os.path.join(get_base_dir(), "token_bytes.pt")
torch.save(token_bytes, token_bytes_path)
print(f"Saved token_bytes to {token_bytes_path}")

Saved token_bytes to /Users/ericsilberstein/.cache/my_nanochat/token_bytes.pt


In [5]:
for i in [65, 500, 20_000]:
    print(f"{token_strings[i:i+5]} -> {token_bytes[i:i+5].tolist()}")

['A', 'B', 'C', 'D', 'E'] -> [1, 1, 1, 1, 1]
[' all', ' U', ' cont', ' dis', ' comp'] -> [4, 2, 5, 4, 5]
[' ubiquitous', 'Rich', ' tails', ' faults', ' disrupted'] -> [11, 4, 6, 7, 10]


In [6]:
!ls -lh {get_base_dir()}

total 2204624
drwxr-xr-x  3 ericsilberstein  staff    96B Nov 11 11:02 base_checkpoints
-rw-r--r--  1 ericsilberstein  staff   826K Oct 28 16:02 my-tokenizer.pkl
-rw-r--r--  1 ericsilberstein  staff    90M Oct 28 10:53 shard_00000.parquet
-rw-r--r--  1 ericsilberstein  staff    90M Oct 28 10:53 shard_00001.parquet
-rw-r--r--  1 ericsilberstein  staff    89M Oct 28 10:53 shard_00002.parquet
-rw-r--r--  1 ericsilberstein  staff    89M Oct 28 10:53 shard_00003.parquet
-rw-r--r--  1 ericsilberstein  staff    91M Oct 28 10:53 shard_00004.parquet
-rw-r--r--  1 ericsilberstein  staff    89M Oct 28 10:53 shard_00005.parquet
-rw-r--r--  1 ericsilberstein  staff    89M Oct 28 10:53 shard_00006.parquet
-rw-r--r--  1 ericsilberstein  staff    89M Oct 28 10:53 shard_00007.parquet
-rw-r--r--  1 ericsilberstein  staff    89M Oct 28 10:53 shard_00008.parquet
-rw-r--r--  1 ericsilberstein  staff    89M Oct 28 10:53 shard_00009.parquet
-rw-r--r--  1 ericsilberstein  staff    90M Nov  1 07:21 shard_00010

Also add `get_token_bytes()` to `my_tokenizer.py` to load the file

In [10]:
get_token_bytes()[20_000:20_005]

tensor([11,  4,  6,  7, 10], dtype=torch.int32)

### Add ability to load a model

To try out evaluation code I'll need a model. I could just instantiate a new one, but I might as well now add `load_checkpoint()` and `build_model()` to `my_checkpoint_manager.py`.

In [2]:
from my_nanochat.my_checkpoint_manager import load_checkpoint, build_model

In [3]:
checkpoint_dir = os.path.join(get_base_dir(), "base_checkpoints", "d4")

In [4]:
model_data, optimizer_data, meta_data = load_checkpoint(checkpoint_dir, step=10, device="cpu")

In [5]:
len(model_data), type(model_data), model_data.keys()

(26,
 collections.OrderedDict,
 odict_keys(['transformer.wte.weight', 'transformer.h.0.attn.c_q.weight', 'transformer.h.0.attn.c_k.weight', 'transformer.h.0.attn.c_v.weight', 'transformer.h.0.attn.c_proj.weight', 'transformer.h.0.mlp.c_fc.weight', 'transformer.h.0.mlp.c_proj.weight', 'transformer.h.1.attn.c_q.weight', 'transformer.h.1.attn.c_k.weight', 'transformer.h.1.attn.c_v.weight', 'transformer.h.1.attn.c_proj.weight', 'transformer.h.1.mlp.c_fc.weight', 'transformer.h.1.mlp.c_proj.weight', 'transformer.h.2.attn.c_q.weight', 'transformer.h.2.attn.c_k.weight', 'transformer.h.2.attn.c_v.weight', 'transformer.h.2.attn.c_proj.weight', 'transformer.h.2.mlp.c_fc.weight', 'transformer.h.2.mlp.c_proj.weight', 'transformer.h.3.attn.c_q.weight', 'transformer.h.3.attn.c_k.weight', 'transformer.h.3.attn.c_v.weight', 'transformer.h.3.attn.c_proj.weight', 'transformer.h.3.mlp.c_fc.weight', 'transformer.h.3.mlp.c_proj.weight', 'lm_head.weight']))

In [6]:
meta_data

{'step': 10,
 'val_bpb': -1,
 'model_config': {'sequence_len': 128,
  'vocab_size': 65537,
  'n_layer': 4,
  'n_head': 2,
  'n_kv_head': 2,
  'n_embd': 256},
 'user_config': {'device_type': '',
  'depth': 4,
  'max_seq_len': 128,
  'num_iterations': 10,
  'device_batch_size': 1,
  'total_batch_size': 128,
  'embedding_lr': 0.2,
  'unembedding_lr': 0.004,
  'weight_decay': 0.0,
  'matrix_lr': 0.02,
  'grad_clip': 1.0,
  'warmup_ratio': 0.0,
  'warmdown_ratio': 0.2,
  'final_lr_frac': 0.0,
  'eval_every': 250,
  'eval_tokens': 10485760,
  'core_metric_every': 2000,
  'core_metric_max_per_task': 500,
  'sample_every': 2000,
  'model_tag': ''},
 'device_batch_size': 1,
 'max_seq_len': 128}

In [3]:
checkpoint_dir = os.path.join(get_base_dir(), "base_checkpoints", "d4")
model, tokenizer, meta_data = build_model(checkpoint_dir, step=10, device=torch.get_default_device(), phase="eval")

Building model with config: {'sequence_len': 128, 'vocab_size': 65537, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}


In [4]:
model

GPT(
  (transformer): ModuleDict(
    (wte): Embedding(65537, 256)
    (h): ModuleList(
      (0-3): 4 x Block(
        (attn): CausalSelfAttention(
          (c_q): Linear(in_features=256, out_features=256, bias=False)
          (c_k): Linear(in_features=256, out_features=256, bias=False)
          (c_v): Linear(in_features=256, out_features=256, bias=False)
          (c_proj): Linear(in_features=256, out_features=256, bias=False)
        )
        (mlp): MLP(
          (c_fc): Linear(in_features=256, out_features=1024, bias=False)
          (c_proj): Linear(in_features=1024, out_features=256, bias=False)
        )
      )
    )
  )
  (lm_head): Linear(in_features=256, out_features=65537, bias=False)
)

In [5]:
tokenizer.get_vocab_size()

65537

### Evaluate BPB

Understand the code in [loss_eval.py](https://github.com/karpathy/nanochat/blob/master/nanochat/loss_eval.py)

He says:

```
    Instead of the naive 'mean loss', this function returns the bits per byte (bpb),
    which is a tokenization vocab size-indepedent metric, meaning you are still comparing
    apples:apples if you change the vocab size. The way this works is that instead of just
    calculating the average loss as usual, you calculate the sum loss, and indepependently
    also the sum bytes (of all the target tokens), and divide. This normalizes the loss by
    the number of bytes that the target tokens represent.

    The added complexity is so that:
    1) All "normal" tokens are normalized by the length of the token in bytes
    2) No special tokens (e.g. <|bos|>) are included in the metric - they are masked out.
    3) No actively masked tokens (using ignore_index of e.g. -1) are included in the metric.

    In addition to evaluate_loss, we need the token_bytes tensor:
    It is a 1D tensor of shape (vocab_size,), indicating the number of bytes for
    each token id, or 0 if the token is to not be counted (e.g. special tokens).
```

The function signature is `def evaluate_bpb(model, batches, steps, token_bytes)`

In [108]:
from my_nanochat.my_checkpoint_manager import build_model
from my_nanochat.my_tokenizer import get_token_bytes
from my_nanochat.my_dataloader import tokenizing_distributed_data_loader
import torch.distributed as dist
import math

First get everything I'll need

In [147]:
device = torch.get_default_device()
checkpoint_dir = os.path.join(get_base_dir(), "base_checkpoints", "d4")
model, tokenizer, meta_data = build_model(checkpoint_dir, step=10, device=device, phase="eval")

Building model with config: {'sequence_len': 128, 'vocab_size': 65537, 'n_layer': 4, 'n_head': 2, 'n_kv_head': 2, 'n_embd': 256}


In [148]:
batches = tokenizing_distributed_data_loader(
    2, # so we can more easily see the batch dimension instead of meta_data["device_batch_size"],
    meta_data["max_seq_len"],
    split="val",
    device=device)

In [149]:
token_bytes = get_token_bytes()

In [150]:
# just for here
# these are eval_steps, how many steps we need to get through the number of tokens we want to use for eval
steps = 1

Now step through the `evaluate_bpb()` function

In [151]:
# nat per token seems to be loss per token but don't really understand it
total_nats = torch.tensor(0.0, dtype=torch.float32, device=model.get_device())
total_bytes = torch.tensor(0.0, dtype=torch.int64, device=model.get_device())
batch_iter = iter(batches)

In [152]:
x.shape

torch.Size([2, 128])

In [153]:
#will be in a loop: for _ in range(steps):
x, y = next(batch_iter)
loss2d = model(x, y, loss_reduction='none') # (B, T) <-- believe wrong, already flat
loss2d = loss2d.view(-1) # flatten
y = y.view(-1) # flatten

In [154]:
if (y.int() < 0).any():
    # he comments:
    # slightly more complex code path if some target tokens are ignore_index (e.g. -1)
    # any target token < 0 is to be ignored: do NOT index token_bytes with negatives
    valid = y >= 0
    y_safe = torch.where(valid, y, torch.zeros_like(y)) # invalids will be 0
    num_bytes2d = torch.where(         # not sure why called num_bytes2d
        valid,
        token_bytes[y_safe],
        torch.zeros_like(y, dtype=token_bytes.dtype))
    total_nats += (loss2d * (num_bytes2d > 0)).sum() # sums loss excluding invalid y
    total_bytes += num_bytes2d.sum()
else:
    num_bytes2d = token_bytes[y]
    total_nats += (loss2d * (num_bytes2d > 0)).sum()
    total_bytes += num_bytes2d.sum()

In [155]:
# nearly there...now some DDP stuff
world_size = dist.get_world_size() if dist.is_initialized() else 1
if world_size > 1:
    dist.all_reduce(total_nats, op=dist.ReduceOp.SUM)
    dist.all_reduce(total_bytes, op=dist.ReduceOp.SUM)

In [156]:
# move both to cpu, calculate bpb and return
total_nats = total_nats.item()
total_bytes = total_bytes.item()
if total_bytes == 0:
    bpb = float('inf')
bpb = total_nats / (math.log(2) * total_bytes)
bpb

2.929899219834058

What's the intuition?

In [158]:
total_nats, total_bytes

(2593.397216796875, 1277)

In [159]:
# so in this case:
total_nats / (math.log(2) * total_bytes)

2.929899219834058

In [160]:
# but let's say these tokens on average had fewer bytes, say more were for single letters and short words
2643 / (math.log(2) * (total_bytes - 200))

3.540429891429462

TODO: come back to better understand this including what a nat is, why this is useful, etc.

In [161]:
# Put it into the function:

@torch.no_grad()
def evaluate_bpb(model, batches, steps, token_bytes):
    total_nats = torch.tensor(0.0, dtype=torch.float32, device=model.get_device())
    total_bytes = torch.tensor(0.0, dtype=torch.int64, device=model.get_device())
    batch_iter = iter(batches)
    for _ in range(steps):
        x, y = next(batch_iter)
        loss2d = model(x, y, loss_reduction='none') # (B, T) <-- believe wrong, already flat
        loss2d = loss2d.view(-1) # flatten
        y = y.view(-1) # flatten
        if (y.int() < 0).any():
            # he comments:
            # slightly more complex code path if some target tokens are ignore_index (e.g. -1)
            # any target token < 0 is to be ignored: do NOT index token_bytes with negatives
            valid = y >= 0
            y_safe = torch.where(valid, y, torch.zeros_like(y)) # invalids will be 0
            num_bytes2d = torch.where(         # not sure why called num_bytes2d
                valid,
                token_bytes[y_safe],
                torch.zeros_like(y, dtype=token_bytes.dtype))
            total_nats += (loss2d * (num_bytes2d > 0)).sum() # sums loss excluding invalid y
            total_bytes += num_bytes2d.sum()
        else:
            num_bytes2d = token_bytes[y]
            total_nats += (loss2d * (num_bytes2d > 0)).sum()
            total_bytes += num_bytes2d.sum()
    # nearly there...now some DDP stuff
    world_size = dist.get_world_size() if dist.is_initialized() else 1
    if world_size > 1:
        dist.all_reduce(total_nats, op=dist.ReduceOp.SUM)
        dist.all_reduce(total_bytes, op=dist.ReduceOp.SUM)
    # move both to cpu, calculate bpb and return
    total_nats = total_nats.item()
    total_bytes = total_bytes.item()
    if total_bytes == 0:
        return float('inf')
    bpb = total_nats / (math.log(2) * total_bytes)
    return bpb

check if it matches

In [162]:
batches = tokenizing_distributed_data_loader(
    2, # so we can more easily see the batch dimension instead of meta_data["device_batch_size"],
    meta_data["max_seq_len"],
    split="val",
    device=device)
evaluate_bpb(model, batches, steps, token_bytes)

2.929899219834058

move it to `my_loss_eval.py`

In [165]:
from my_nanochat.my_loss_eval import evaluate_bpb

In [167]:
evaluate_bpb.__module__

'my_nanochat.my_loss_eval'

In [168]:
batches = tokenizing_distributed_data_loader(
    2, # so we can more easily see the batch dimension instead of meta_data["device_batch_size"],
    meta_data["max_seq_len"],
    split="val",
    device=device)
evaluate_bpb(model, batches, steps, token_bytes)

2.929899219834058

### Add evaluate BPB to training

Update `my_base_train.py`, now try it:

In [173]:
import os
os.environ["PYTHONPATH"] = "../my_nanochat"

In [176]:
!python -m scripts.my_base_train \
    --depth=4 \
    --max_seq_len=128 \
    --device_batch_size=1 \
    --num_iterations=20 \
    --total_batch_size=128 \
    --eval_every=5 \
    --eval_tokens=1280

overriding depth = 4
overriding max_seq_len = 128
overriding device_batch_size = 1
overriding num_iterations = 20
overriding total_batch_size = 128
overriding eval_every = 5
overriding eval_tokens = 1280
user_config: {'device_type': '', 'depth': 4, 'max_seq_len': 128, 'num_iterations': 20, 'device_batch_size': 1, 'total_batch_size': 128, 'embedding_lr': 0.2, 'unembedding_lr': 0.004, 'weight_decay': 0.0, 'matrix_lr': 0.02, 'grad_clip': 1.0, 'warmup_ratio': 0.0, 'warmdown_ratio': 0.2, 'final_lr_frac': 0.0, 'eval_every': 5, 'eval_tokens': 1280, 'core_metric_every': 2000, 'core_metric_max_per_task': 500, 'sample_every': 2000, 'model_tag': ''}
Autodetected device type: mps
Vocab size: 65,537
num_layers: 4
model_dim: 256
num_heads: 2
num_kv_heads: 2
Tokens / micro-batch / rank: 1 x 128 = 128
Tokens / micro-batch: 128
Total batch size 128 => gradient accumulation steps: 1
GPT(
  (transformer): ModuleDict(
    (wte): Embedding(65537, 256)
    (h): ModuleList(
      (0-3): 4 x Block(
        (a

^ See the validation bpb printed on every 5th step and the min printed at the end

Code added as part of this challenge:

- Added `build_model()` to `my_checkpoint_manager.py` so can load saved checkpoints

- Added `my_loss_eval.py` where the BPB stuff lives

- Added `get_token_bytes()` to `my_tokenizer.py` to read the saved token ids -> byte count file **but did not** yet move the code to compute the mapping out of this notebook

- Updated `my_base_train.py` to do BPB eval